# IPC (Information Processing Capacity) Demo

Demonstrates computing network capacity using Legendre basis and Ridge regression.
Based on Dambre et al. (2012) methodology.

**Pipeline**:
1. Build small network with 3 areas
2. Drive network with random input signal
3. Compute IPC using polynomial degrees 0-3
4. Report total capacity and per-degree breakdown
5. Verify capacity ≤ area size (sanity check)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from pyac.core.network import Network
from pyac.core.rng import make_rng
from pyac.core.types import AreaSpec, FiberSpec, NetworkSpec
from pyac.measures.ipc.runner import run_ipc

In [ ]:
# Build small network for fast execution
network_spec = NetworkSpec(
    areas=[
        AreaSpec(name="input", n=100, k=10),
        AreaSpec(name="hidden", n=200, k=20),
        AreaSpec(name="output", n=150, k=15),
    ],
    fibers=[
        FiberSpec(src="input", dst="hidden", p_fiber=0.2),
        FiberSpec(src="hidden", dst="output", p_fiber=0.2),
    ],
    beta=0.01
)

network = Network(network_spec, make_rng(42))

print(f"Network: {len(network_spec.areas)} areas, {len(network_spec.fibers)} fibers")
for area in network_spec.areas:
    print(f"  - {area.name}: n={area.n}, k={area.k}")

In [ ]:
# Run IPC pipeline
# runner.py generates random input signal internally
print("Computing IPC...")

results = run_ipc(
    network=network,
    area_name="output",
    max_degree=3,
    max_delay=5,
    signal_length=500,  # Shorter for fast execution
    rng=make_rng(100)
)

print("IPC computation complete")

In [ ]:
# Report results and verify sanity checks
area_size = network_spec.areas[2].n  # "output" area
total_capacity = results['total_capacity']
degree_capacities = results['degree_breakdown']

print("\n=== IPC Results ===")
print(f"Total capacity: {total_capacity:.2f}")
print(f"Area size: {area_size}")
print(f"Sanity check (capacity ≤ area size): {total_capacity <= area_size}")

print(f"\nPer-degree capacities:")
for deg in sorted(degree_capacities.keys()):
    cap = degree_capacities[deg]
    print(f"  Degree {deg}: {cap:.2f}")
    assert cap >= 0, f"Negative capacity at degree {deg}"

print("\nAll sanity checks passed!")

In [ ]:
# Plot capacity vs polynomial degree
plt.figure(figsize=(10, 6))
degrees = sorted(degree_capacities.keys())
capacities = [degree_capacities[d] for d in degrees]

plt.bar(degrees, capacities, color='#2E86AB', alpha=0.8, label='Capacity per degree')
plt.axhline(y=area_size, color='#E63946', linestyle='--', 
            linewidth=1.5, label=f'Area size ({area_size})')

plt.xlabel('Polynomial Degree', fontsize=12)
plt.ylabel('Capacity', fontsize=12)
plt.title('IPC: Capacity vs Polynomial Degree', fontsize=14)
plt.grid(True, alpha=0.3, axis='y')
plt.legend(fontsize=11)
plt.xticks(degrees)
plt.tight_layout()
plt.show()

print("\n=== Notebook Execution Complete ===")